In [ ]:
import os
import numpy as np
import networkx as nx

# Force geopandas to use Shapely 2.0 instead of PyGEOS
# (PyGEOS was merged with Shapely, and PyGEOS on its own will stop working in a future release of GeoPandas)
os.environ['USE_PYGEOS'] = '0' 

import geopandas as gpd
import osmnx as ox

# Generate a graph

In [ ]:
MDG = ox.graph.graph_from_point((32.231774, -110.943864), 
                                dist=1000, # meters from center
                                dist_type='bbox', 
                                network_type='drive', 
                                simplify=False, 
                                retain_all=True, 
                                truncate_by_edge=False, 
                                clean_periphery=False, 
                                custom_filter=None)

# Remove interstitial nodes (nodes that are not intersections or dead-ends)
MDG = ox.simplification.simplify_graph(MDG, 
                                       strict=False, 
                                       remove_rings=False, 
                                       track_merged=False)

print(type(MDG))
print(nx.is_weighted(MDG, weight='length'))

The graph `MDG` that is generated is a graph type in NetworkX that is called a **MultiDiGraph**, which means that it is a directed graph that allows for self-loops and parallel edges.

In [ ]:
MG = ox.utils_graph.get_undirected(MDG)
print(type(MG))
print(nx.is_weighted(MG, weight='length'))

In [ ]:
G = MG

In [ ]:
ox.plot.plot_graph(G)

# Find routes

In [ ]:
import os
os.chdir('/home/jovyan/work/src')

In [ ]:
import edge
import route
import route_finder
import graph_utils

In [ ]:
from importlib import reload

reload(edge)
reload(route)
reload(route_finder)
reload(graph_utils)

In [ ]:
source = ox.distance.nearest_nodes(G, -110.943864, 32.231774, return_dist=False)
max_distance = 700

In [ ]:
rf = route_finder.RouteFinder(G, source, max_distance)
nx.set_node_attributes(rf.G, values=False, name='visited')

In [ ]:
route = rf.greedy_nearest()

In [ ]:
print('route = ',route)
print('edges = ',route.edges)
print('number of unique nodes in route = ',len(set(route.nodes)))

In [ ]:
rf = route_finder.RouteFinder(G, source, max_distance)
nx.set_node_attributes(rf.G, values=False, name='visited')

brute_force_routes = rf.brute_force()

In [ ]:
len(brute_force_routes)

In [ ]:
num_new_nodes=[]
for r in brute_force_routes:
    num_new_nodes.append(len(set(r.nodes)))
max(num_new_nodes)


# Visualize the route on a map

In [ ]:
# Plot the route on a map

fig, ax = ox.plot.plot_graph_route(G, route.nodes, 
                                   route_color='r', route_linewidth=4, route_alpha=0.5, 
                                   orig_dest_size=100, 
                                   ax=None)

In [ ]:
# Make a crude animation of the route

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

fig, ax = ox.plot_graph(G, show=False, close=False)

def update(i):
    # replot things
    ax.scatter(G.nodes[route.nodes[i]]['x'], G.nodes[route.nodes[i]]['y'])

animation = FuncAnimation(fig, update, frames=len(route.nodes), interval=500)

animation.save('/home/jovyan/work/notebooks/route.gif', writer='pillow')

# Plot using folium

Figure out how to plot the route using folium.
Plotting in `osmnx` can produce some lines that aren't edges, and I don't know why.

In [ ]:
import folium

In [ ]:
fmap = ox.folium.plot_graph_folium(G, graph_map=None, popup_attribute=None,
                                  tiles='cartodbpositron', zoom=1, fit_bounds=True,
                                  color='gray', weight=2)

fmap

In [ ]:
gdf_nodes, gdf_edges = ox.utils_graph.graph_to_gdfs(G, nodes=True, edges=True, 
                                                    node_geometry=True, fill_edge_geometry=True)

In [ ]:
osmids = gdf_nodes.index.to_numpy()
for n in osmids:
    
    if n in set(route.nodes):
        color = 'blue'
    else:
        color = 'red'
        
    y = gdf_nodes.loc[n].y
    x = gdf_nodes.loc[n].x
    
    folium.Circle(location=[y, x], 
                  radius=10, # in meters
                  color=color,
                  opacity=0.5, 
                  weight=1, 
                  fill=True
                 ).add_to(fmap)
    

fmap

In [ ]:
for e in route.edges:
    try:
        linestring = gdf_edges.loc[(e[0], e[1], e[2])]['geometry']
    except KeyError:
        linestring = gdf_edges.loc[(e[1], e[0], e[2])]['geometry']

    polyline = ox.folium._make_folium_polyline(linestring, color='blue')
    polyline.add_to(fmap)

fmap